# Automated ML

TODO: Import Dependencies. In the cell below, import all the dependencies that you will need to complete the project.

In [1]:
from azureml.core import Experiment, Model, Webservice
from azureml.core.workspace import Workspace
from azureml.core.dataset import Dataset
from azureml.train.automl import AutoMLConfig
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException
from azureml.pipeline.steps import AutoMLStep

import pandas as pd

## Dataset

### Overview
TODO: In this markdown cell, give an overview of the dataset you are using. Also mention the task you will be performing.
We will be using the 'Credit Card customers' dataset from Kaggle and predicting if a customer is likely to churn.

TODO: Get data. In the cell below, write code to access the data you will be using in this project. Remember that the dataset needs to be external.

In [2]:
ws = Workspace.from_config()

# Choose a name for experiment
experiment_name = 'demo-automl-experiment'

experiment=Experiment(ws, experiment_name)

# Prepare the datastore to upload data
datastore = ws.get_default_datastore()

# Set the upload location to target_path in datastore
datastore.upload(src_dir='./', target_path = 'data')

Performing interactive authentication. Please follow the instructions on the terminal.
To sign in, use a web browser to open the page https://microsoft.com/devicelogin and enter the code AVA9WRGQH to authenticate.
You have logged in. Now let us find all the subscriptions to which you have access...
Interactive authentication successfully completed.
Uploading an estimated of 8 files
Uploading ./automl.ipynb
Uploaded ./automl.ipynb, 1 files out of an estimated total of 8
Uploading ./automl.ipynb.amltemp
Uploaded ./automl.ipynb.amltemp, 2 files out of an estimated total of 8
Uploading ./hyperparameter_tuning.ipynb
Uploaded ./hyperparameter_tuning.ipynb, 3 files out of an estimated total of 8
Uploading ./hyperparameter_tuning.ipynb.amltemp
Uploaded ./hyperparameter_tuning.ipynb.amltemp, 4 files out of an estimated total of 8
Uploading ./score.py
Uploaded ./score.py, 5 files out of an estimated total of 8
Uploading ./train.py
Uploaded ./train.py, 6 files out of an estimated total of 8
Uploa

$AZUREML_DATAREFERENCE_2bdf48f6681f442888cf78d97fd01631

In [3]:
# Choose a name for CPU cluster
compute_name = "demo-cluster"

# Verify that cluster does not already exist
try:
    compute_target = ComputeTarget(workspace=ws, name=compute_name)
    print('Found existing cluster, use it.')
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_D2_V2',# for GPU, use "STANDARD_NC6"
                                                           #vm_priority = 'lowpriority', # optional
                                                           max_nodes=4)
    compute_target = ComputeTarget.create(ws, compute_name, compute_config)

compute_target.wait_for_completion(show_output=True)

Found existing cluster, use it.

Running


In [4]:
dataset = Dataset.get_by_name(ws, name = "bankchurners")
dataset

{
  "source": [
    "('workspaceblobstore', 'UI/01-28-2021_121111_UTC/BankChurners.csv')"
  ],
  "definition": [
    "GetDatastoreFiles",
    "ParseDelimited",
    "DropColumns",
    "SetColumnTypes"
  ],
  "registration": {
    "id": "293d15e3-7f22-4872-90c2-2c83a4835e5a",
    "name": "bankchurners",
    "version": 1,
    "workspace": "Workspace.create(name='quick-starts-ws-136373', subscription_id='d4ad7261-832d-46b2-b093-22156001df5b', resource_group='aml-quickstarts-136373')"
  }
}

## AutoML Configuration

TODO: Explain why you chose the automl settings and cofiguration you used below.

In [5]:
# List the automl config
automl_config = AutoMLConfig(
    experiment_timeout_minutes = 30,
    task = "classification",
    compute_target = "demo-cluster",
    max_concurrent_iterations = 5,
    training_data = dataset,
    primary_metric = "accuracy",
    label_column_name = "Attrition_Flag",
    n_cross_validations = 3,
    enable_early_stopping = True
)

In [6]:
# Submit the experiment
remote_run = experiment.submit(automl_config, show_output = True)

Running on remote.
No run_configuration provided, running on demo-cluster with default configuration
Running on remote compute: demo-cluster
Parent Run ID: AutoML_3ba117a6-e744-4727-ba4f-d123f69c864d

Current status: FeaturesGeneration. Generating features for the dataset.
Current status: DatasetFeaturization. Beginning to fit featurizers and featurize the dataset.
Current status: DatasetCrossValidationSplit. Generating individually featurized CV splits.
Current status: ModelSelection. Beginning model selection.

****************************************************************************************************
DATA GUARDRAILS: 

TYPE:         Class balancing detection
STATUS:       ALERTED
DESCRIPTION:  To decrease model bias, please cancel the current run and fix balancing problem.
              Learn more about imbalanced data: https://aka.ms/AutomatedMLImbalancedData
DETAILS:      Imbalanced data can lead to a falsely perceived positive effect of a model's accuracy because the inp

## Run Details

OPTIONAL: Write about the different models trained and their performance. Why do you think some models did better than others?

TODO: In the cell below, use the `RunDetails` widget to show the different experiments.

In [7]:
from azureml.widgets import RunDetails
RunDetails(remote_run).show()
remote_run.wait_for_completion(show_output = True)

_AutoMLWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', 's…



****************************************************************************************************
DATA GUARDRAILS: 

TYPE:         Class balancing detection
STATUS:       ALERTED
DESCRIPTION:  To decrease model bias, please cancel the current run and fix balancing problem.
              Learn more about imbalanced data: https://aka.ms/AutomatedMLImbalancedData
DETAILS:      Imbalanced data can lead to a falsely perceived positive effect of a model's accuracy because the input data has bias towards one class.
+---------------------------------+---------------------------------+--------------------------------------+
|Size of the smallest class       |Name/Label of the smallest class |Number of samples in the training data|
+=================================+=================================+======================================+
|1627                             |Attrited Customer                |10127                                 |
+---------------------------------+----------

{'runId': 'AutoML_3ba117a6-e744-4727-ba4f-d123f69c864d',
 'target': 'demo-cluster',
 'status': 'Completed',
 'startTimeUtc': '2021-01-28T12:17:48.144838Z',
 'endTimeUtc': '2021-01-28T12:36:08.537692Z',
 'properties': {'num_iterations': '1000',
  'training_type': 'TrainFull',
  'acquisition_function': 'EI',
  'primary_metric': 'accuracy',
  'train_split': '0',
  'acquisition_parameter': '0',
  'num_cross_validation': '3',
  'target': 'demo-cluster',
  'AMLSettingsJsonString': '{"path":null,"name":"demo-automl-experiment","subscription_id":"d4ad7261-832d-46b2-b093-22156001df5b","resource_group":"aml-quickstarts-136373","workspace_name":"quick-starts-ws-136373","region":"southcentralus","compute_target":"demo-cluster","spark_service":null,"azure_service":"remote","many_models":false,"pipeline_fetch_max_batch_size":1,"iterations":1000,"primary_metric":"accuracy","task_type":"classification","data_script":null,"validation_size":0.0,"n_cross_validations":3,"y_min":null,"y_max":null,"num_clas

## Best Model

TODO: In the cell below, get the best model from the automl experiments and display all the properties of the model.



In [8]:
# Retrieve the best AutoML model
best_run, best_model = remote_run.get_output()
best_metrics = best_run.get_metrics()

In [9]:
#Save the best model
# model = best_run.register_model(model_name = "best-automl-model", model_path = "./outputs/")
model = best_run.register_model(model_name='best-automl-model',model_path='outputs/model.pkl', tags={'Method':'AutoML'}, properties={'accuracy': best_metrics['accuracy']})
print(model)
print(best_run)

Model(workspace=Workspace.create(name='quick-starts-ws-136373', subscription_id='d4ad7261-832d-46b2-b093-22156001df5b', resource_group='aml-quickstarts-136373'), name=best-automl-model, id=best-automl-model:1, version=1, tags={'Method': 'AutoML'}, properties={'accuracy': '0.9708698730325903'})
Run(Experiment: demo-automl-experiment,
Id: AutoML_3ba117a6-e744-4727-ba4f-d123f69c864d_38,
Type: azureml.scriptrun,
Status: Completed)


In [10]:
print(best_metrics["accuracy"])
print(best_model._final_estimator)

0.9708698730325903
PreFittedSoftVotingClassifier(classification_labels=None,
                              estimators=[('0',
                                           Pipeline(memory=None,
                                                    steps=[('maxabsscaler',
                                                            MaxAbsScaler(copy=True)),
                                                           ('lightgbmclassifier',
                                                            LightGBMClassifier(boosting_type='gbdt',
                                                                               class_weight=None,
                                                                               colsample_bytree=1.0,
                                                                               importance_type='split',
                                                                               learning_rate=0.1,
                                                                

## Model Deployment

Remember you have to deploy only one of the two models you trained.. Perform the steps in the rest of this notebook only if you wish to deploy this model.

TODO: In the cell below, register the model, create an inference config and deploy the model as a web service.

In [11]:
from azureml.core import Environment
from azureml.core.model import InferenceConfig
from azureml.automl.core.shared import constants

best_run.download_file('./outputs/model.pkl', './outputs/bankchurners-auto-model.pkl')
best_run.download_file('./outputs/scoring_file_v_1_0_0.py', './outputs/score_aml.py')
best_run.download_file('./automl_driver.py', './outputs/automl_driver.py')
best_run.download_file(constants.CONDA_ENV_FILE_PATH, 'myenv.yml')

myenv = Environment.from_conda_specification(name="myenv", file_path="myenv.yml")

# Create the inference config

inference_config = InferenceConfig(entry_script='./score.py', environment=myenv)


In [12]:
f = open("envnew.yml", "w")
f.write(myenv.python.conda_dependencies.serialize_to_string())
f.close()

print("packages", myenv.python.conda_dependencies.serialize_to_string())

packages # Conda environment specification. The dependencies defined in this file will
# be automatically provisioned for runs with userManagedDependencies=False.

# Details about the Conda environment file format:
# https://conda.io/docs/user-guide/tasks/manage-environments.html#create-env-file-manually

name: project_environment
dependencies:
  # The python interpreter version.
  # Currently Azure ML only supports 3.5.2 and later.
- python=3.6.2

- pip:
  - azureml-train-automl-runtime==1.20.0.post1
  - inference-schema
  - azureml-interpret==1.20.0
  - azureml-defaults==1.20.0
- numpy>=1.16.0,<1.19.0
- pandas==0.25.1
- scikit-learn==0.22.1
- py-xgboost<=0.90
- fbprophet==0.5
- holidays==0.9.11
- psutil>=5.2.2,<6.0.0
channels:
- anaconda
- conda-forge



In [13]:
# Deploy the model as a webservice
from azureml.core.webservice import AciWebservice

deployment_config = AciWebservice.deploy_configuration(
    cpu_cores=1,
    memory_gb = 1,
    enable_app_insights = True
)
service = Model.deploy(ws, 'model-service', [model], inference_config, deployment_config)

print(service.state)

# Wait until deployment is complete
service.wait_for_deployment(show_output = True)


Transitioning
Tips: You can try get_logs(): https://aka.ms/debugimage#dockerlog or local deployment: https://aka.ms/debugimage#debug-locally to debug if deployment takes longer than 10 minutes.
Running.............................................................................................................................................................................................
Succeeded
ACI service creation operation finished, operation "Succeeded"


In [14]:
print(service.get_logs)

<bound method Webservice.get_logs of AciWebservice(workspace=Workspace.create(name='quick-starts-ws-136373', subscription_id='d4ad7261-832d-46b2-b093-22156001df5b', resource_group='aml-quickstarts-136373'), name=model-service, image_id=None, compute_type=None, state=ACI, scoring_uri=Healthy, tags=http://e872b7b8-b699-4242-a9c1-82b7bf985738.southcentralus.azurecontainer.io/score, properties={}, created_by={'hasInferenceSchema': 'True', 'hasHttps': 'False'})>


TODO: In the cell below, send a request to the web service you deployed to test it.

In [18]:
import json

# Importing the data set for testing 

sample_data = dataset.to_pandas_dataframe().sample(10)
data_sample = sample_data.drop("Attrition_Flag", axis=1)
test_sample = json.dumps({"data": data_sample.to_dict()})

print(test_sample)

{"data": {"CLIENTNUM": {"2726": 716375883, "7916": 713782608, "10122": 772366833, "9732": 778563858, "2674": 803408733, "345": 716984658, "1220": 718726833, "6632": 780567408, "7008": 771499233, "4927": 779468808}, "Customer_Age": {"2726": 53, "7916": 46, "10122": 50, "9732": 40, "2674": 32, "345": 49, "1220": 39, "6632": 47, "7008": 46, "4927": 43}, "Gender": {"2726": "F", "7916": "F", "10122": "M", "9732": "F", "2674": "M", "345": "M", "1220": "M", "6632": "F", "7008": "F", "4927": "M"}, "Dependent_count": {"2726": 3, "7916": 4, "10122": 2, "9732": 3, "2674": 2, "345": 1, "1220": 1, "6632": 3, "7008": 4, "4927": 1}, "Education_Level": {"2726": "Uneducated", "7916": "Graduate", "10122": "Graduate", "9732": "Uneducated", "2674": "High School", "345": "Graduate", "1220": "High School", "6632": "Graduate", "7008": "High School", "4927": "High School"}, "Marital_Status": {"2726": "Married", "7916": "Single", "10122": "Single", "9732": "Married", "2674": "Divorced", "345": "Married", "1220

In [23]:
import requests
scoring_uri = 'http://e872b7b8-b699-4242-a9c1-82b7bf985738.southcentralus.azurecontainer.io/score'
headers = {'Content-type': 'application/json'}

response = requests.post(scoring_uri, test_sample, headers=headers)

In [24]:
# Print results from the inference
print(response.status_code)
print(response.text)

200
"{\"result\": [\"Existing Customer\", \"Existing Customer\", \"Existing Customer\", \"Existing Customer\", \"Existing Customer\", \"Existing Customer\", \"Existing Customer\", \"Existing Customer\", \"Existing Customer\", \"Existing Customer\"]}"


TODO: In the cell below, print the logs of the web service and delete the service

In [ ]:
# Print web service logs
service_logs = service.get_logs()

for line in service_logs.split('\n'):
    print(line)

# Delete the service
service.delete()